In [6]:

conda install -c conda-forge folium

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import io
import json # library to handle JSON files
!pip install geocoder==1.5.0
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!pip install geocoder==1.5.0

Solving environment: ...working... done

# All requested packages already installed.



usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Get content of the Wiki page with Soup library

In [8]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

Get content of the table row elements from the page. Get only text from the inside of the element.
Each new line separate by "\n" element (= new line) and each column by coma.

In [9]:
tdElements =  soup.findAll("td")
cleantext = BeautifulSoup(str(tdElements), "lxml").get_text()
my_list = cleantext.split("\n")
myTable = []
for row in my_list:
    myTable.append(row[1:].split(","))

Convert array to DataFrame. Add column names

In [10]:
df = pd.DataFrame(myTable)
df.columns = ["PostalCode", "Borough", "Neighborhood"]

Drop rows where we don't have values or Borough is not assigned.Strip from whitespaces the values of the cells in order to have string match. Make copy in order to modify new dataframe

In [11]:
df_noNA = df.dropna()
df_noNotAssigned =  df_noNA[df_noNA["Borough"].str.strip() != 'Not assigned'].copy()

Copy values from Borough column when Neighborhood column has 'Not assigned' value. Group Neighborhood values by PostalCode and Borough, since we want to preserve 3 columns. Display the content of the dataframe

In [12]:
df_noNotAssigned.Neighborhood = np.where(df_noNotAssigned.Neighborhood == 'Not assigned', df_noNotAssigned.Borough, df_noNotAssigned.Neighborhood)
df_grouped = df_noNotAssigned.groupby(["PostalCode","Borough"] ,as_index=False)["Neighborhood"].agg(lambda x: "%s" % ', '.join(x))
df_grouped

PostalCode       Borough  \
0          M1B   Scarborough   
1          M1C   Scarborough   
2          M1E   Scarborough   
3          M1G   Scarborough   
4          M1H   Scarborough   
..         ...           ...   
98         M9N          York   
99         M9P     Etobicoke   
100        M9R     Etobicoke   
101        M9V     Etobicoke   
102        M9W     Etobicoke   

                                          Neighborhood  
0                                      Rouge,  Malvern  
1             Highland Creek,  Rouge Hill,  Port Union  
2                  Guildwood,  Morningside,  West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100   Kingsview Village,  Martin Grove Gardens,  Ri...  
101   Albion Gardens,  Beaumond Heights,  Humbergat...  
102                                          Northwest  

[103 rows x 3 columns]

In [13]:
df_grouped.shape

(103, 3)

Read from csv

In [14]:
coords = pd.read_csv("Geospatial_Coordinates.csv")

Rename column to match the one from the dataframe. Remove whitespace characters in order to be able to merge by content

In [15]:
coords = coords.rename(columns={'Postal Code':'PostalCode'})
coords.PostalCode =  coords["PostalCode"].str.strip()
df_grouped.PostalCode =  df_grouped["PostalCode"].str.strip()

In [16]:
result = pd.merge(df_grouped, coords, how='left')
result.head()

PostalCode       Borough                               Neighborhood  \
0        M1B   Scarborough                            Rouge,  Malvern   
1        M1C   Scarborough   Highland Creek,  Rouge Hill,  Port Union   
2        M1E   Scarborough        Guildwood,  Morningside,  West Hill   
3        M1G   Scarborough                                     Woburn   
4        M1H   Scarborough                                  Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

In [17]:
!pip install geopy

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="TR_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
result

PostalCode       Borough  \
0          M1B   Scarborough   
1          M1C   Scarborough   
2          M1E   Scarborough   
3          M1G   Scarborough   
4          M1H   Scarborough   
..         ...           ...   
98         M9N          York   
99         M9P     Etobicoke   
100        M9R     Etobicoke   
101        M9V     Etobicoke   
102        M9W     Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                      Rouge,  Malvern  43.806686 -79.194353  
1             Highland Creek,  Rouge Hill,  Port Union  43.784535 -79.160497  
2                  Guildwood,  Morningside,  West Hill  43.763573 -79.188711  
3                                               Woburn  43.770992 -79.216917  
4                                            Cedarbrae  43.773136 -79.239476  
..                                                 ...        ...        ...  
98                                              Weston  43.706876 -79.518188  
99                                           Westmount  43.696319 -79.532242  
100   Kingsview Village,  Martin Grove Gardens,  Ri...  43.688905 -79.554724  
101   Albion Gardens,  Beaumond Heights,  Humbergat...  43.739416 -79.588437  
102                                          Northwest  43.706748 -79.594054  

[103 rows x 5 columns]

In [21]:
scar_data = result[result['Borough'] == ' Scarborough'].reset_index(drop=True)
scar_data

PostalCode       Borough  \
0         M1B   Scarborough   
1         M1C   Scarborough   
2         M1E   Scarborough   
3         M1G   Scarborough   
4         M1H   Scarborough   
5         M1J   Scarborough   
6         M1K   Scarborough   
7         M1L   Scarborough   
8         M1M   Scarborough   
9         M1N   Scarborough   
10        M1P   Scarborough   
11        M1R   Scarborough   
12        M1S   Scarborough   
13        M1T   Scarborough   
14        M1V   Scarborough   
15        M1W   Scarborough   
16        M1X   Scarborough   

                                         Neighborhood   Latitude  Longitude  
0                                     Rouge,  Malvern  43.806686 -79.194353  
1            Highland Creek,  Rouge Hill,  Port Union  43.784535 -79.160497  
2                 Guildwood,  Morningside,  West Hill  43.763573 -79.188711  
3                                              Woburn  43.770992 -79.216917  
4                                           Cedarbrae  43.773136 -79.239476  
5                                 Scarborough Village  43.744734 -79.239476  
6       East Birchmount Park,  Ionview,  Kennedy Park  43.727929 -79.262029  
7                   Clairlea,  Golden Mile,  Oakridge  43.711112 -79.284577  
8    Cliffcrest,  Cliffside,  Scarborough Village ...  43.716316 -79.239476  
9                        Birch Cliff,  Cliffside West  43.692657 -79.264848  
10   Dorset Park,  Scarborough Town Centre,  Wexfo...  43.757410 -79.273304  
11                                 Maryvale,  Wexford  43.750072 -79.295849  
12                                          Agincourt  43.794200 -79.262029  
13          Clarks Corners,  Sullivan,  Tam O'Shanter  43.781638 -79.304302  
14   Agincourt North,  L'Amoreaux East,  Milliken,...  43.815252 -79.284577  
15                                    L'Amoreaux West  43.799525 -79.318389  
16                                        Upper Rouge  43.836125 -79.205636

In [23]:
print('The dataframe has {} boroughs and {} Neighborhood.'.format(
        len(scar_data['Borough'].unique()),
        scar_data.shape[0]
    )
)

The dataframe has 1 boroughs and 17 Neighborhood.


In [24]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="TR_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
CLIENT_ID = 'ZCNA3TD1ZS3WWHGUDVSF4CKVO5SB5SA0IED4YOWXEOI4SKXO' # My Foursquare ID
CLIENT_SECRET = 'JVE0AK4OIJJEBNAGCCCXPT5345NDOKQ3JWKRSYHDNFPKH4NE' # My Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZCNA3TD1ZS3WWHGUDVSF4CKVO5SB5SA0IED4YOWXEOI4SKXO
CLIENT_SECRET:JVE0AK4OIJJEBNAGCCCXPT5345NDOKQ3JWKRSYHDNFPKH4NE


In [28]:
scar_data.loc[0, 'Neighborhood']

' Rouge,  Malvern'

In [29]:
neighborhood_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scar_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Rouge,  Malvern are 43.806686299999996, -79.19435340000001.


In [30]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=ZCNA3TD1ZS3WWHGUDVSF4CKVO5SB5SA0IED4YOWXEOI4SKXO&client_secret=JVE0AK4OIJJEBNAGCCCXPT5345NDOKQ3JWKRSYHDNFPKH4NE&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7d1fe6df2774001bc412d5'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\SergiuH\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


name            categories        lat        lng
0  Wendy’s  Fast Food Restaurant  43.807448 -79.199056

In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )

 Rouge,  Malvern
 Highland Creek,  Rouge Hill,  Port Union
 Guildwood,  Morningside,  West Hill
 Woburn
 Cedarbrae
 Scarborough Village
 East Birchmount Park,  Ionview,  Kennedy Park
 Clairlea,  Golden Mile,  Oakridge
 Cliffcrest,  Cliffside,  Scarborough Village West
 Birch Cliff,  Cliffside West
 Dorset Park,  Scarborough Town Centre,  Wexford Heights
 Maryvale,  Wexford
 Agincourt
 Clarks Corners,  Sullivan,  Tam O'Shanter
 Agincourt North,  L'Amoreaux East,  Milliken,  Steeles East
 L'Amoreaux West
 Upper Rouge


In [37]:
print(scar_venues.shape)
scar_venues.head()

(88, 7)


Neighborhood  Neighborhood Latitude  \
0                            Rouge,  Malvern              43.806686   
1   Highland Creek,  Rouge Hill,  Port Union              43.784535   
2   Highland Creek,  Rouge Hill,  Port Union              43.784535   
3        Guildwood,  Morningside,  West Hill              43.763573   
4        Guildwood,  Morningside,  West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497      RIGHT WAY TO GOLF       43.785177   
2              -79.160497  Royal Canadian Legion       43.782533   
3              -79.188711      G & G Electronics       43.765309   
4              -79.188711             Marina Spa       43.766000   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.161108           Golf Course  
2       -79.163085                   Bar  
3       -79.191537     Electronics Store  
4       -79.191000                   Spa

In [38]:
#how many venues were returned for each neighborhood
scar_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
 Agincourt                                                              4   
 Agincourt North,  L'Amoreaux East,  Milliken, ...                      2   
 Birch Cliff,  Cliffside West                                           4   
 Cedarbrae                                                              8   
 Clairlea,  Golden Mile,  Oakridge                                     10   
 Clarks Corners,  Sullivan,  Tam O'Shanter                             13   
 Cliffcrest,  Cliffside,  Scarborough Village West                      3   
 Dorset Park,  Scarborough Town Centre,  Wexfor...                      7   
 East Birchmount Park,  Ionview,  Kennedy Park                          4   
 Guildwood,  Morningside,  West Hill                                    7   
 Highland Creek,  Rouge Hill,  Port Union                               2   
 L'Amoreaux West                                                       14   
 Maryvale,  Wexford                                                     4   
 Rouge,  Malvern                                                        1   
 Scarborough Village                                                    1   
 Woburn                                                                 4   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
 Agincourt                                                               4   
 Agincourt North,  L'Amoreaux East,  Milliken, ...                       2   
 Birch Cliff,  Cliffside West                                            4   
 Cedarbrae                                                               8   
 Clairlea,  Golden Mile,  Oakridge                                      10   
 Clarks Corners,  Sullivan,  Tam O'Shanter                              13   
 Cliffcrest,  Cliffside,  Scarborough Village West                       3   
 Dorset Park,  Scarborough Town Centre,  Wexfor...                       7   
 East Birchmount Park,  Ionview,  Kennedy Park                           4   
 Guildwood,  Morningside,  West Hill                                     7   
 Highland Creek,  Rouge Hill,  Port Union                                2   
 L'Amoreaux West                                                        14   
 Maryvale,  Wexford                                                      4   
 Rouge,  Malvern                                                         1   
 Scarborough Village                                                     1   
 Woburn                                                                  4   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
 Agincourt                                              4               4   
 Agincourt North,  L'Amoreaux East,  Milliken, ...      2               2   
 Birch Cliff,  Cliffside West                           4               4   
 Cedarbrae                                              8               8   
 Clairlea,  Golden Mile,  Oakridge                     10              10   
 Clarks Corners,  Sullivan,  Tam O'Shanter             13              13   
 Cliffcrest,  Cliffside,  Scarborough Village West      3               3   
 Dorset Park,  Scarborough Town Centre,  Wexfor...      7               7   
 East Birchmount Park,  Ionview,  Kennedy Park          4               4   
 Guildwood,  Morningside,  West Hill                    7               7   
 Highland Creek,  Rouge Hill,  Port Union               2               2   
 L'Amoreaux West                                       14              14   
 Maryvale,  Wexford                                     4               4   
 Rouge,  Malvern                                        1               1   
 Scarborough Village        

In [40]:
#how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 53 uniques categories.


In [41]:

# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

Neighborhood  American Restaurant  \
0                            Rouge,  Malvern                    0   
1   Highland Creek,  Rouge Hill,  Port Union                    0   
2   Highland Creek,  Rouge Hill,  Port Union                    0   
3        Guildwood,  Morningside,  West Hill                    0   
4        Guildwood,  Morningside,  West Hill                    0   

   Athletics & Sports  Bakery  Bank  Bar  Breakfast Spot  Brewery  Bus Line  \
0                   0       0     0    0               0        0         0   
1                   0       0     0    0               0        0         0   
2                   0       0     0    1               0        0         0   
3                   0       0     0    0               0        0         0   
4                   0       0     0    0               0        0         0   

   Bus Station  ...  Pizza Place  Playground  Rental Car Location  \
0            0  ...            0           0                    0   
1            0  ...            0           0                    0   
2            0  ...            0           0                    0   
3            0  ...            0           0                    0   
4            0  ...            0           0                    0   

   Sandwich Place  Skating Rink  Smoke Shop  Soccer Field  Spa  \
0               0             0           0             0    0   
1               0             0           0             0    0   
2               0             0           0             0    0   
3               0             0           0             0    0   
4               0             0           0             0    1   

   Thai Restaurant  Vietnamese Restaurant  
0                0                      0  
1                0                      0  
2                0                      0  
3                0                      0  
4                0                      0  

[5 rows x 54 columns]

In [42]:
scar_onehot.shape

(88, 54)

In [43]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

Neighborhood  American Restaurant  \
0                                           Agincourt             0.000000   
1    Agincourt North,  L'Amoreaux East,  Milliken,...             0.000000   
2                        Birch Cliff,  Cliffside West             0.000000   
3                                           Cedarbrae             0.000000   
4                   Clairlea,  Golden Mile,  Oakridge             0.000000   
5           Clarks Corners,  Sullivan,  Tam O'Shanter             0.000000   
6    Cliffcrest,  Cliffside,  Scarborough Village ...             0.333333   
7    Dorset Park,  Scarborough Town Centre,  Wexfo...             0.000000   
8       East Birchmount Park,  Ionview,  Kennedy Park             0.000000   
9                 Guildwood,  Morningside,  West Hill             0.000000   
10           Highland Creek,  Rouge Hill,  Port Union             0.000000   
11                                    L'Amoreaux West             0.000000   
12                                 Maryvale,  Wexford             0.000000   
13                                    Rouge,  Malvern             0.000000   
14                                Scarborough Village             0.000000   
15                                             Woburn             0.000000   

    Athletics & Sports  Bakery      Bank  Bar  Breakfast Spot   Brewery  \
0                0.000   0.000  0.000000  0.0        0.250000  0.000000   
1                0.000   0.000  0.000000  0.0        0.000000  0.000000   
2                0.000   0.000  0.000000  0.0        0.000000  0.000000   
3                0.125   0.125  0.125000  0.0        0.000000  0.000000   
4                0.000   0.200  0.000000  0.0        0.000000  0.000000   
5                0.000   0.000  0.076923  0.0        0.000000  0.000000   
6                0.000   0.000  0.000000  0.0        0.000000  0.000000   
7                0.000   0.000  0.000000  0.0        0.000000  0.142857   
8                0.000   0.000  0.000000  0.0        0.000000  0.000000   
9                0.000   0.000  0.142857  0.0        0.000000  0.000000   
10               0.000   0.000  0.000000  0.5        0.000000  0.000000   
11               0.000   0.000  0.071429  0.0        0.071429  0.000000   
12               0.000   0.250  0.000000  0.0        0.250000  0.000000   
13               0.000   0.000  0.000000  0.0        0.000000  0.000000   
14               0.000   0.000  0.000000  0.0        0.000000  0.000000   
15               0.000   0.000  0.000000  0.0        0.000000  0.000000   

    Bus Line  Bus Station  ...  Pizza Place  Playground  Rental Car Location  \
0        0.0          0.0  ...     0.000000         0.0             0.000000   
1        0.0          0.0  ...     0.000000         0.5             0.000000   
2        0.0          0.0  ...     0.000000         0.0             0.000000   
3        0.0          0.0  ...     0.000000         0.0             0.000000   
4        0.2          0.1  ...     0.000000         0.0             0.000000   
5        0.0          0.0  ...     0.153846         0.0             0.000000   
6        0.0          0.0  ...     0.000000         0.0             0.000000   
7        0.0          0.0  ...     0.000000         0.0             0.000000   
8        0.0          0.0  ...     0.000000         0.0             0.000000   
9        0.0          0.0  ...     0.000000         0.0             0.142857   
10       0.0          0.0  ...     0.000000         0.0             0.000000   
11       0.0          0.0  ...     0.071429         0.0             0.000000   
12       0.0          0.0  ...     0.000000         0.0             0.000000   
13       0.0          0.0  ...     0.000000         0.0             0.000000   
14       0.0          0.0  ...     0.000000         1.0             0.000000   
15       0.0          0.0  ...     0.000000         0.0             0.000000   

    Sandwich Place  Skating Rink  Smoke Shop  Soccer Field       Spa  \
0    

In [44]:
scar_grouped.shape

(16, 54)

In [45]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


---- Agincourt North,  L'Amoreaux East,  Milliken,  Steeles East----
                venue  freq
0                Park   0.5
1          Playground   0.5
2        Noodle House   0.0
3  Italian Restaurant   0.0
4   Korean Restaurant   0.0


---- Birch Cliff,  Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


---- Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.12
1               Bakery  0.12
2                 Bank  0.12
3      Thai Restaurant  0.12
4  Fried Chicken Joint  0.12


---- Clairlea,  Golden Mile,  Oakridge----
            venue  freq
0        Bus Line   0.2
1          Bakery 

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
#display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                          Agincourt          Skating Rink   
1   Agincourt North,  L'Amoreaux East,  Milliken,...            Playground   
2                       Birch Cliff,  Cliffside West       College Stadium   
3                                          Cedarbrae       Thai Restaurant   
4                  Clairlea,  Golden Mile,  Oakridge                Bakery   

   2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0         Breakfast Spot  Latin American Restaurant                Lounge   
1                   Park      Vietnamese Restaurant           Coffee Shop   
2  General Entertainment               Skating Rink                  Café   
3     Athletics & Sports                     Bakery                  Bank   
4               Bus Line             Ice Cream Shop          Soccer Field   

   5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0  Vietnamese Restaurant       College Stadium   General Entertainment   
1  General Entertainment           Gas Station  Furniture / Home Store   
2  Vietnamese Restaurant           Golf Course             Gas Station   
3            Gas Station   Fried Chicken Joint    Caribbean Restaurant   
4           Intersection           Bus Station                    Park   

    8th Most Common Venue   9th Most Common Venue 10th Most Common Venue  
0             Gas Station  Furniture / Home Store    Fried Chicken Joint  
1     Fried Chicken Joint    Fast Food Restaurant      Electronics Store  
2  Furniture / Home Store     Fried Chicken Joint   Fast Food Restaurant  
3        Hakka Restaurant        Department Store          Grocery Store  
4           Metro Station       Convenience Store            Gas Station

In [62]:
#Clustering analysis

# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 2, 0, 0, 0])

In [64]:
# add clustering labels


scar_merged = scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

PostalCode       Borough                               Neighborhood  \
0        M1B   Scarborough                            Rouge,  Malvern   
1        M1C   Scarborough   Highland Creek,  Rouge Hill,  Port Union   
2        M1E   Scarborough        Guildwood,  Morningside,  West Hill   
3        M1G   Scarborough                                     Woburn   
4        M1H   Scarborough                                  Cedarbrae   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.806686 -79.194353             4.0  Fast Food Restaurant   
1  43.784535 -79.160497             3.0           Golf Course   
2  43.763573 -79.188711             0.0                   Spa   
3  43.770992 -79.216917             0.0           Coffee Shop   
4  43.773136 -79.239476             0.0       Thai Restaurant   

   2nd Most Common Venue  3rd Most Common Venue  4th Most Common Venue  \
0  Vietnamese Restaurant       Hakka Restaurant            Golf Course   
1                    Bar  Vietnamese Restaurant        College Stadium   
2                   Bank    Rental Car Location           Intersection   
3      Korean Restaurant               Pharmacy  Vietnamese Restaurant   
4     Athletics & Sports                 Bakery                   Bank   

   5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0  General Entertainment           Gas Station  Furniture / Home Store   
1  General Entertainment           Gas Station  Furniture / Home Store   
2      Electronics Store        Medical Center      Mexican Restaurant   
3  General Entertainment           Gas Station  Furniture / Home Store   
4            Gas Station   Fried Chicken Joint    Caribbean Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0    Fried Chicken Joint     Electronics Store         Discount Store  
1    Fried Chicken Joint  Fast Food Restaurant      Electronics Store  
2  Vietnamese Restaurant       College Stadium            Gas Station  
3    Fried Chicken Joint  Fast Food Restaurant      Electronics Store  
4       Hakka Restaurant      Department Store          Grocery Store

In [70]:
scar_merged.dropna(subset = ["Cluster Labels"], inplace=True)
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].astype(int)

In [71]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
#Cluster1
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  2nd Most Common Venue  \
2    Scarborough               0                   Spa                   Bank   
3    Scarborough               0           Coffee Shop      Korean Restaurant   
4    Scarborough               0       Thai Restaurant     Athletics & Sports   
6    Scarborough               0           Coffee Shop         Discount Store   
7    Scarborough               0                Bakery               Bus Line   
9    Scarborough               0       College Stadium  General Entertainment   
10   Scarborough               0     Indian Restaurant  Vietnamese Restaurant   
11   Scarborough               0                Bakery             Smoke Shop   
12   Scarborough               0          Skating Rink         Breakfast Spot   
13   Scarborough               0              Pharmacy            Pizza Place   
15   Scarborough               0  Fast Food Restaurant            Coffee Shop   

        3rd Most Common Venue      4th Most Common Venue  \
2         Rental Car Location               Intersection   
3                    Pharmacy      Vietnamese Restaurant   
4                      Bakery                       Bank   
6            Department Store          Convenience Store   
7              Ice Cream Shop               Soccer Field   
9                Skating Rink                       Café   
10                    Brewery     Furniture / Home Store   
11             Breakfast Spot  Middle Eastern Restaurant   
12  Latin American Restaurant                     Lounge   
13       Fast Food Restaurant         Italian Restaurant   
15         Chinese Restaurant                 Nail Salon   

    5th Most Common Venue   6th Most Common Venue   7th Most Common Venue  \
2       Electronics Store          Medical Center      Mexican Restaurant   
3   General Entertainment             Gas Station  Furniture / Home Store   
4             Gas Station     Fried Chicken Joint    Caribbean Restaurant   
6        Hakka Restaurant             Golf Course   General Entertainment   
7            Intersection             Bus Station                    Park   
9   Vietnamese Restaurant             Golf Course             Gas Station   
10              Pet Store      Chinese Restaurant          Breakfast Spot   
11  Vietnamese Restaurant         College Stadium   General Entertainment   
12  Vietnamese Restaurant         College Stadium   General Entertainment   
13     Chinese Restaurant            Noodle House       Convenience Store   
15            Pizza Place  Furniture / Home Store                Pharmacy   

     8th Most Common Venue   9th Most Common Venue 10th Most Common Venue  
2    Vietnamese Restaurant         College Stadium            Gas Station  
3      Fried Chicken Joint    Fast Food Restaurant      Electronics Store  
4         Hakka Restaurant        Department Store          Grocery Store  
6              Gas Station  Furniture / Home Store    Fried Chicken Joint  
7            Metro Station       Convenience Store            Gas Station  
9   Furniture / Home Store     Fried Chicken Joint   Fast Food Restaurant  
10        Department Store      Athletics & Sports  General Entertainment  
11             Gas Station  Furniture / Home Store    Fried Chicken Joint  
12             Gas Station  Furniture / Home Store    Fried Chicken Joint  
13     Fried Chicken Joint                    Bank        Thai Restaurant  
15           Grocery Store          Sandwich Place                   Bank

In [74]:
#Cluster2
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  2nd Most Common Venue  \
5    Scarborough               1            Playground  Vietnamese Restaurant   
14   Scarborough               1            Playground                   Park   

    3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
5             Coffee Shop           Golf Course  General Entertainment   
14  Vietnamese Restaurant           Coffee Shop  General Entertainment   

   6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
5            Gas Station  Furniture / Home Store   Fried Chicken Joint   
14           Gas Station  Furniture / Home Store   Fried Chicken Joint   

   9th Most Common Venue 10th Most Common Venue  
5   Fast Food Restaurant      Electronics Store  
14  Fast Food Restaurant      Electronics Store

In [75]:
#Cluster3
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
8   Scarborough               2                 Motel   American Restaurant   

  3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
8       College Stadium           Golf Course  General Entertainment   

  6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
8           Gas Station  Furniture / Home Store   Fried Chicken Joint   

  9th Most Common Venue 10th Most Common Venue  
8  Fast Food Restaurant      Electronics Store

In [76]:
#Cluster4
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
1   Scarborough               3           Golf Course                   Bar   

   3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
1  Vietnamese Restaurant       College Stadium  General Entertainment   

  6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
1           Gas Station  Furniture / Home Store   Fried Chicken Joint   

  9th Most Common Venue 10th Most Common Venue  
1  Fast Food Restaurant      Electronics Store

In [77]:
#Cluster5
scar_merged.loc[scar_merged['Cluster Labels'] == 4, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  2nd Most Common Venue  \
0   Scarborough               4  Fast Food Restaurant  Vietnamese Restaurant   

  3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
0      Hakka Restaurant           Golf Course  General Entertainment   

  6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
0           Gas Station  Furniture / Home Store   Fried Chicken Joint   

  9th Most Common Venue 10th Most Common Venue  
0     Electronics Store         Discount Store